# Reading Jupyter notebooks into Python

For my [digitized notes](https://jessicastringham.net/2018/05/06/notebook-tour.html) project, I wrote a few scripts that read Markdown cells from Jupyter notebook files. Specifically, I read a notebook's non-empty Markdown cells and used them for my search index and flashcard database. Reading Jupyter notebooks as data is pretty easy! Below I'll read the non-empty markdown cells.

In [1]:
import nbformat

path = '2018-05-02-HMM.ipynb'

NB_VERSION = 4

with open(path) as f:
    nb = nbformat.read(f, NB_VERSION)

markdown_cells = [
    cell['source']
    for cell in nb['cells']  # go through the cells
    if cell['cell_type'] == 'markdown' and cell['source']  # skip things like 'code' cells, and empty markdown cells
]

## Displaying Markdown

Below shows how to render markdown in a iPython notebook to show what I can do with a dictionary of Jupyter notebook data. This is also how I render flashcards in [digitized notes](https://jessicastringham.net/2018/05/06/notebook-tour.html).

In [2]:
from IPython.display import display, Markdown

display(Markdown("**Below is data loaded from [this other file]({})!** \n\n".format(path)))
display(Markdown(markdown_cells[0]))

**Below is data loaded from [this other file](2018-05-02-HMM.ipynb)!** 



# HMM exact inference

This demo shows exact inference on a HMM with known, discrete transition and emission distributions that are fixed over time. It does alpha recursion, which is a sum-product algorithm on HMMs.

The problem is taken from Example 23.3 from ["Bayesian Reasoning and Machine Learning"](http://www.cs.ucl.ac.uk/staff/d.barber/brml/), and this is more or less a port of the provided Matlab implementation with my commentary.

I'm trying to understand how my probabilistic graphical modelling class can be represented in code, so I try to go through this painfully slow and show how the `numpy` arrays map to the discrete probability distributions.

## Problem

The idea is that there's someone moving around a room. You can't see them, but you can hear bumps and creaks from them moving around. You know the probability of someone bumping or creaking at each location in the room. You also know how one moves around the room.
Using the sequence of bumps and/or creaks, the goal is to figure out the locations in the room.

### HMMs

![HMM diagram](2018-05-02-hmm.png)

This problem can be modeled using a Hidden Markov Model. At each timestep $t$ in the sequence, there's the visible state $v_t$ (the bump/creak combination) and the hidden state $h_t$ (the location in the room.)

The goal of *filtering* is to show the probability distribution over the locations in the room at a given timestep $t$ given the bumps/creaks from the current and previous timesteps, $p(h_t\mid v_{1:t})$. Plotting this distribution at each timestep gives a heatmap of where in the room we think the person is. 

In this model, there is a transition distribution $p(h_t \mid h_{t - 1})$ to show that each hidden state depends on the previous timestep's hidden state. There also is the emission distribution $p(v_t \mid h_t)$ to show each visible state depends on the corresponding hidden state. There's also the probability of where in the room the person starts, $p(h_1)$.

The rest of the notebook shows:
 - Setting up the known distributions $p(h_1)$, $p(h_t \mid h_{t - 1})$, and $p(v_t \mid h_t)$. This is a lot of the notebook!
 - Generating some data for the locations ($h_{1:t}$) and sounds ($v_{1:t}$) from that distribution.
 - Finally I get to alpha recursion. I try to predict $h_{1:t}$ based on $v_{1:t}$, $p(h_1)$, $p(h_t \mid h_{t - 1})$, and $p(v_t \mid h_t)$.